# Table of Contents
 - [Starting with basics](#starting-with-basics)
   - [Install Modules](#install-modules)
   - [Manage Permissions](#manage-permissions)
   - [Get Email](#get-email)
   - [Send Teams Message](#send-teams-message)
 - [Combining these for complex uses](#combining-these-for-complex-uses)
   - [Run a Report](#run-a-report)
   - [Summarize Report](#summarize-report)
   - [Send Report Summary to Teams](#send-report-summary-to-teams)

In [35]:
flowchart TB
    subgraph one [Starting with basics]
        direction LR
        s1(Install Modules) ==> s2(Manage Permissions)
        s2 ==> s3(Get Email)
        s3 ==> s4(Send Teams Message)
    end
    subgraph two [Combining these for complex uses]
        direction LR
        s5(Run a Report) ==> s6(Summarize Report)
        s6 ==> s7(Send Report to Teams)
    end
    one ==> two

# Starting with basics

## Install Modules

In [ ]:
$modules = @(
    "Microsoft.Graph.Authentication",
    "Microsoft.Graph.Mail",
    "Microsoft.Graph.Teams",
    "Microsoft.Graph.Reports"
)
foreach($module in $modules)
{
    Install-Module $module
}

## Manage Permissions

In [ ]:
$permissions = @()
$commands = @("Get-MgUserMessage","New-MgChatMessage","Get-MgReportEmailActivityUserDetail")
foreach ($command in $commands)
{
    $p = (Find-MgGraphCommand -Command $command).Permissions.Name
    $permissions += $p | Select-Object -Unique
    $permissions = $permissions | Select-Object -Unique | Sort-Object
}
Connect-MgGraph -Scopes $permissions

## Get Email

In [ ]:
$messages = Get-MgUserMessage -UserId "admin@M365x64319768.onmicrosoft.com"
$messages | Select-Object -First 1 @{n='From';e={$_.From.EmailAddress.Address}},Subject,BodyPreview

## Send Teams Message

In [ ]:
$chatId = "19:Y0smUUwArynK0X-hfNKJeoVb_JymRgAs80oAu94qqlA1@thread.tacv2"
$message = @{
    ContentType = "html"
    Content = @"
        <h2>New email message</h2>
        <p><b>From:</b> $(($messages|Select-Object -First 1).From.EmailAddress.Address)</p>
        <p><b>To:</b> $(($messages|Select-Object -First 1).ToRecipients.EmailAddress.Address)</p>
        <p><b>Subject:</b> $(($messages|Select-Object -First 1).Subject)</p>
        <p><b>Body Preview:</b> $(($messages|Select-Object -First 1).BodyPreview)</p>
"@
}
New-MgChatMessage -ChatId "$chatId"`
    -Body $message

# Combining these for complex uses

## Run a Report

In [ ]:
Get-MgReportEmailActivityUserDetail -Period "D30" -OutFile .\temp.csv

## Summarize Report

In [ ]:
$report = Import-Csv .\temp.csv
$summary = $report|Measure-Object 'Send Count','Receive Count','Read Count','Meeting Created Count','Meeting Interacted Count' -Average -Sum -Maximum -Minimum

Check formating...

In [ ]:
$summary|select Property,Count,Sum,Maximum,Minimum,Average|ConvertTo-Html -Fragment

## Send Report Summary to Teams

In [ ]:
$message = @{
    ContentType = "html"
    Content = @"
        <h2>Monthly Email Utilization Report</h2>
        <p>
        $($summary|Select-Object Property,Count,Sum,Maximum,Minimum,Average|ConvertTo-Html -Fragment)
        </p>
"@
}
New-MgChatMessage -ChatId "$chatId"`
    -Body $message